# Requeriments

In [1]:
#!python.exe -m pip install --upgrade pip
#!pip install selenium
#!pip install telegram
#!pip install python-telegram-bot

# Imports

In [4]:
import time
import yaml
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from datetime import datetime
import pandas as pd
import os
import logging
from logging.config import dictConfig
from ftplib import FTP
import telegram
from telegram import Update
from telegram.ext import Updater, CommandHandler
from telegram.ext.callbackcontext import CallbackContext
import warnings

# Setup

In [5]:
warnings.filterwarnings("ignore", category=DeprecationWarning) 

stream = open("config.yaml", 'r')
c = yaml.safe_load(stream)

#Dito
DITO_EMAIL = c['email']
DITO_SENHA = c['senha'] 
api_key = c['api_key']
api_secret = c['api_secret']

TELEGRAM_BOT_TOKEN = c['telegram_api']
TELEGRAM_CHAT_ID = c['bot_chat_id'] 

DRIVER_PATH = '/path/to/chromedriver'
options = Options()
options.headless = True
options.add_argument("--window-size=1920,1080")
timestamp = int(datetime.utcnow().timestamp())
logging.basicConfig(filename=f"log/log_{timestamp}.log", level=logging.INFO)

Output_Directory = "/retorno_dito"

driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)

bot = telegram.Bot(token=TELEGRAM_BOT_TOKEN)

In [6]:
def login_ftp():
    ftp = FTP(c['ftp_host'])
    ftp.login(c['ftp_user'],c['ftp_password'])
    return ftp
    

In [8]:
def telegram_bot_sendtext(bot_message, num_try = 0):
    global bot
    try:
        bot = telegram.Bot(token=TELEGRAM_BOT_TOKEN)
        return bot.send_message(chat_id=TELEGRAM_CHAT_ID, text=bot_message)
    except:
        if num_try == 1:
            bot = telegram.Bot(token=TELEGRAM_BOT_TOKEN)
            return telegram_bot_sendtext(bot_message, 1)
        return 0

In [9]:
login_page = 'https://crm.dito.com.br/admins/sign_in'
notification_page = 'https://crm.dito.com.br/notifications'
report_page = ''

login_path = '//*[@id="admin_email"]'
password_path = '//*[@id="admin_password"]'
login_button = '//*[@id="new_admin"]/div[4]/input'

menu_date_path = '//*[@id="notifications-calendar"]/span[2]'
menu_date_today = '//*[@id="period-datepicker"]/div[1]/span[1]'
menu_date_yesterday = '//*[@id="period-datepicker"]/div[1]/span[2]'
menu_date_mtd = '//*[@id="period-datepicker"]/div[1]/span[3]'
menu_date_last_month = '//*[@id="period-datepicker"]/div[1]/span[4]'
menu_date_last_7d = '//*[@id="period-datepicker"]/div[1]/span[5]'
menu_date_last_15d = '//*[@id="period-datepicker"]/div[1]/span[6]'
menu_date_last_30d = '//*[@id="period-datepicker"]/div[1]/span[7]'
menu_date_last_90d = '//*[@id="period-datepicker"]/div[1]/span[8]'

filter_button_path = '//*[@id="notifications-calendar"]/span[2]'
export_button_path = '//*[@id="page-actions"]/button[2]'
confirm_export_path = '//*[@id="export-warning"]/div[2]/div/div/button'

export_path = 'https://crm.dito.com.br/exports'
notifications_path = '//*[@id="notifications"]'
download_button_1 = '//*[@id="exports"]/div/table/tbody/tr[2]/td[4]/a'

csv_content_path = '/html/body/pre/text()'

# ETL

In [7]:
try:
    logging.info('[INFO] - Starting ETL.')
    telegram_bot_sendtext('🤖 INITIALIZING ETL - DITO 🔔')

    logging.info(f'[INFO] - Beginning Extraction step.')
    logging.info(f'[INFO] - Starting Selenium.')
    try:
        driver.get(login_page)
        #driver.save_screenshot('debug/login_1.png')
        login = driver.find_element(By.XPATH, login_path).send_keys(DITO_EMAIL)
        password = driver.find_element(By.XPATH, password_path).send_keys(DITO_SENHA)
        time.sleep(1)
        #driver.save_screenshot('debug/login_2.png')
        submit = driver.find_element(By.XPATH,login_button).click()
        time.sleep(1)
        #driver.save_screenshot('debug/login_3.png')
        time.sleep(5)
        logging.info('[INFO] - Login successful.')
    except:
        telegram_bot_sendtext('🤖 LOGIN ERROR ⚠️')
        driver.quit()
        exit()
    
    try:
        driver.get(notification_page)
        time.sleep(3)
        menu_date = driver.find_element(By.XPATH, menu_date_path).click()
        #driver.save_screenshot('debug/notification_1.png')
        #mudar o parâmetro para o período selecionado
        time.sleep(1)
        menu_date_pick = driver.find_element(By.XPATH, menu_date_last_15d).click()
        #driver.save_screenshot('debug/notification_2.png')
        time.sleep(1)
        #driver.save_screenshot('debug/notification_3.png')
        time.sleep(1)
        export_button = driver.find_element(By.XPATH, export_button_path).click()
        #driver.save_screenshot('debug/notification_4.png')
        time.sleep(1)
        confirm_export_button = driver.find_element(By.XPATH, confirm_export_path).click()
        timestamp = int(datetime.utcnow().timestamp())
        #driver.save_screenshot('debug/notification_5.png')
        logging.info('[INFO] - Filters set, generating file...')
        time.sleep(60*3)
        logging.info('[INFO] - 6 min passed, begin export...')
    except:
        telegram_bot_sendtext('🤖 ERROR WHILE FILTERING DATA ⚠️')
        driver.quit()
        exit()

    try:
        driver.get(export_path)
        #driver.save_screenshot('debug/export_1.png')
        time.sleep(4)
        open_notification_tab = driver.find_element(By.XPATH, notifications_path).click()
        #driver.save_screenshot('debug/export_2.png')
        time.sleep(2)
        open_last_file = driver.find_element(By.XPATH, download_button_1).click()
        get_url = driver.current_url
        url = get_url.split("U/")
        url = str(url[1])
        logging.info(f'[INFO] - File export successful, generating txt file. Current URL:{url}')
        #driver.save_screenshot('debug/export_3.png')
        time.sleep(2)
        logging.info('[INFO] - File export successful, generating txt file.')
    except:
        telegram_bot_sendtext('🤖 ERROR ON FILE EXPORT ⚠️')
        driver.quit()
        exit()

    try:
        ext_text = driver.find_element(By.TAG_NAME,"body").text
        with open(f'download/{url}.txt','w', encoding='utf-8', newline='\n') as file:
            writer = file.write(ext_text)
            logging.info(f'[INFO] - Txt file generated, {url}.txt')
    except:
        telegram_bot_sendtext('🤖 ERROR WHILE WRITING FILE ⚠️')
        driver.quit()
        exit()

    driver.quit()

    logging.info(f'[INFO] - Extraction completed, Selenium closed.')

    #TRANSFORMAÇÃO TXT PARA CSV
    try: 
        logging.info(f'[INFO] - Beginning Transformation step.')
        data = open(f'download/{url}.txt', 'r', encoding='utf-8').read()
        data = data.encode(encoding="cp1252",errors="ignore").decode('utf8',errors="ignore")
        with open(f'download/{url}_2.txt','w', encoding='utf-8', newline='\n') as file:
            writer = file.write(data)
        df = pd.read_csv(f'download/{url}_2.txt', delimiter = ",", encoding='utf-8')
        df.set_index('Nome', inplace= True)
        df.drop(columns = df.columns[0], axis=1, inplace=True)
        logging.info(f'[INFO] - Dataframe generated.')
        df.to_csv(f'output/{url}', sep=',')
    except:
        telegram_bot_sendtext('🤖 ERROR WHILE TRANSFORMING(ENCODE/DECODE) FILE ⚠️')

    logging.info(f'[INFO] - Transformation completed.')

    logging.info(f'[INFO] - Beginning Loading step.')
    #CARGA NO FTP
    try:
        ftp = login_ftp()
        File2Send= (f'output/{url}')
        logging.info(f'[INFO] - Sending file to FTP. (filename = output/{url}.csv')
        ftp.cwd(Output_Directory)
        with open(File2Send, "rb") as f:
            ftp.storbinary('STOR ' + os.path.basename(File2Send), f) 
            logging.info(f'[INFO] - File sent.')
            telegram_bot_sendtext('🤖 FILE SENT TO FTP ✅')
    except:
        telegram_bot_sendtext('🤖 ERRO NO ENVIO PARA O FTP ⚠️')

    logging.info(f'[INFO] - Loading completed.') 

    logging.info(f'[INFO] - ETL completed.')
    telegram_bot_sendtext('🤖 ETL FINISHED ✅')
except:
    telegram_bot_sendtext('🚨 ETL ERROR 🚨')

# Results

[Result](/assets/results.jpg "Results in FTP")

As we can see above, this are the ETL result. The file are correctly stored in the customer FTP as requested.